In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import numpy as np
from torch.nn import Parameter

from torch.optim import Adam

from get_data import FrameDataset
from treeqn import TreeQN

In [2]:
path = r'C:\Users\mikef\Desktop\BiB Work\Heim_Transformers\BIB-VT-main\bib_train_preprocess'

test = FrameDataset(path, 'cuda', types=['preference'], mode='train', process_data=0)

reading files of type preference in train
9982


In [3]:
input_shape = torch.zeros(1,3,84,84).shape
num_actions = 4
tree_depth = 3
embedding_dim = 10
td_lambda = 0.9

model = TreeQN(input_shape=input_shape, num_actions=num_actions, tree_depth=tree_depth, embedding_dim=embedding_dim, td_lambda=td_lambda)
optimizer = Adam(model.parameters(), lr=0.001)

In [17]:
(test.__getitem__(0)[1][0][0]).shape #get a single frame of a trial, need to unsqueeze to put into model

torch.Size([3, 84, 84])

In [19]:
test_tensor = test.__getitem__(0)[1][0]
q_values, next_states = model(test_tensor[0].unsqueeze(0))
print("q value shape: ", q_values.shape)
print("next state shape: ", next_states.shape)

q value shape:  torch.Size([1, 4])
next state shape:  torch.Size([4, 3, 84, 84])


In [20]:
train_data = 1000
test_data = 1500
#sgd predicting next frame
def train():
    for i in range(train_data):
        episode = test.__getitem__(i)[1]
        for trial in episode:
            if len(trial) < 2:
                continue
            for frame in range(len(trial)-1):
                optimizer.zero_grad()
                input = trial[frame].unsqueeze(0)
                target = trial[frame+1].unsqueeze(0)

                output_q_values, first_transition = model(input)
                #loss is each predicted next state - target next state, weighted by q values of each action
                loss = torch.abs(first_transition - target) #subtract predicted next
                loss = loss.sum(dim=2) #sum along the z axis, we now have 4 action x batch_size sums
                
                loss = loss.transpose(0,1) #transpose to batch_size x 4 action
                output_q_values = output_q_values.unsqueeze(-1)

                #weigh by q values
                loss = loss * output_q_values

                #loss = sum of each action's weighted loss
                #loss should go to zero as accuracy of prediction of next state q values increases
                loss = loss.sum(dim=1)
                loss = loss.sum()
                loss.backward()
                optimizer.step()
train()





In [23]:
def tester():
    for i in range(train_data, test_data):
        episode = test.__getitem__(i)[1]
        for trial in episode:
            if len(trial) < 2:
                continue
            for frame in range(len(trial)-1):
                input = trial[frame].unsqueeze(0)
                target = trial[frame+1].unsqueeze(0)

                output_q_values, first_transition = model(input)
                loss = torch.abs(first_transition - target)
                loss = loss.sum(dim=2)
                loss = loss.transpose(0,1)
                output_q_values = output_q_values.unsqueeze(-1)
                loss = loss * output_q_values
                loss = loss.sum(dim=1)
                loss = loss.sum()
                print(np.round(loss.detach().numpy(),2))
                
                #break for now 
                return 0

In [24]:
tester()

-4.8694095e+24


0